In [1]:
! pip install sqlalchemy
! pip install PyMySQL

You should consider upgrading via the 'c:\users\xabi\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\xabi\anaconda3\python.exe -m pip install --upgrade pip' command.


In [8]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


7. Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

In [54]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


In [55]:
data_may = pd.read_sql_query('SELECT * FROM rentals_may', engine)
data_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [56]:
data_june = pd.read_sql_query('SELECT * FROM rentals_june', engine)
data_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


7.a. Check the number of rentals for each customer for May

In [57]:
may="""
SELECT customer_id,COUNT(rental_id) as number_of_rentals
FROM rentals_may
GROUP BY customer_id;
"""
df_rental_may = pd.read_sql_query("SELECT customer_id,COUNT(rental_id) as number_of_rentals FROM rentals_may GROUP BY customer_id", engine)
df_rental_may.head()

,customer_id,number_of_rentals
0,130,2
1,459,1
2,408,3
3,333,1
4,222,5


In [58]:
df_rental_may=df_rental_may.rename(columns={'number_of_rentals':'rentals_may'})
df_rental_may.head(2)

,customer_id,rentals_may
0,130,2
1,459,1


In [59]:
df_rental_may.shape

(520, 2)

7.b. Check the number of rentals for each customer for June

In [60]:
june="""
SELECT customer_id,COUNT(rental_id) as number_of_rentals
FROM rentals_june
GROUP BY customer_id;
"""
df_rental_june = pd.read_sql_query(june, engine)
df_rental_june.head()

,customer_id,number_of_rentals
0,416,5
1,516,6
2,239,5
3,285,3
4,310,6


In [61]:
df_rental_june=df_rental_june.rename(columns={'number_of_rentals':'rentals_june'})
df_rental_june.head(2)

,customer_id,rentals_june
0,416,5
1,516,6


In [62]:
df_rental_june.shape

(590, 2)

In [63]:
df_merged = pd.merge(df_rental_june, df_rental_may, on='customer_id', how='inner')
df_merged.head(2)

,customer_id,rentals_june,rentals_may
0,416,5,3
1,516,6,2


8. Write a function that checks if customer borrowed more or less films in the month of June as compared to May

In [64]:
def rentals(df_merged):
    if df_merged['rentals_june']>df_merged['rentals_may']:
        rental='More'
        return rental
    
    elif df_merged['rentals_june']<df_merged['rentals_may']:
        rental='Less'
        return rental
    
    else:
        rental='Same'
        return rental

In [65]:
df_merged['june_or_may']=df_merged.apply(rentals,axis=1)
df_merged.head()

,customer_id,rentals_june,rentals_may,june_or_may
0,416,5,3,More
1,516,6,2,More
2,239,5,6,Less
3,310,6,1,More
4,49,5,4,More
